<a href="https://colab.research.google.com/github/Ctrl408/Empirically-Derived-Scaling-Principles-for-an-increasingly-complex-binary-decision-task/blob/main/File2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import numpy as np
from numpy import vstack
import itertools 
import torch 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch import nn
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Tanh
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import BCELoss
import sklearn 
from sklearn.datasets import make_classification 
from sklearn.model_selection import train_test_split 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from google.colab import drive
drive.mount('/content/drive')

#Generate DataSet 
class DataSet:
  
  def data(n=100,d=5,W=None):
   
    global samples 
    samples=list()
    n_1=0
    n_0=0
    n_samples=False

    while n_samples==False:
      A=np.random.uniform(low=0,high=1,size=d)
      A_pow=list()
      for i in range(1,(d+1)):
        A_pow.append(A[(i-1)]**(i))   
      pdt= np.dot(W.T,np.array(A_pow))
    
      if pdt >0:
        if (n_1)==(n/2):pass
        else:
          samples.append([A,1])
          n_1+=1
      else:
        if (n_0)==(n/2):pass
        else:
          samples.append([A,-1])
          n_0+=1
      if len(samples)==n:
        n_samples=True
    
    X,Y=list(),list()  
    for i in samples:      
      X.append(i[0].tolist())
      Y.append(i[1])
    X=np.array(X)
    Y=np.array(Y)
   
    X1,X2,Y1,Y2=train_test_split(X,Y, test_size=.20, random_state=42, shuffle=True, stratify=None)    
    X1,Y1=torch.from_numpy(X1).type(torch.float32),torch.from_numpy(Y1).type(torch.float32)
    dset=TensorDataset(X1,Y1) 
    trainloader=torch.utils.data.DataLoader(dset, batch_size=60, shuffle=False, num_workers=1)
    X2,Y2=torch.from_numpy(X2).type(torch.float32),torch.from_numpy(Y2).type(torch.float32)      
    dset2=TensorDataset(X2,Y2)           
    testloader=torch.utils.data.DataLoader(dset2, batch_size=60, shuffle=False, num_workers=1)
    
    return trainloader,testloader 


class Net(Module):
  
  #MLP with adjustable width and depth 
  def MLP(width,depth): 
       
      layers = []
      width_l=list()
      for i in range(1,width):width_l.append(i)
      for wdth,dpth in zip(width_l,depth):
        if wdth==1:
          prev_wdth=1
          prev_dpth=dpth
          continue 
        layers.append(nn.Linear(prev_dpth,dpth))          
        layers.append(nn.ReLU())
        prev_dpth=dpth
      layers.pop()
      layers.append(Sigmoid())
      global net
      net = nn.Sequential(*layers)
      print(net)
      return net

  def forward(x):
    return net(x)

class Main:

  def __init__(self):
    pass

  def Train(train_dl, model):
    
      Loss_Function= BCELoss()
      optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
      #Training Loop
      #total epochs=1000
      for epoch in range(1000):
          for i, (inputs, targets) in enumerate(train_dl):
              inputs, targets=inputs.to(device), targets.to(device)
              optimizer.zero_grad()            
              yhat = model(inputs)                      
              loss = Loss_Function(yhat, (targets.unsqueeze(1)))            
              loss.backward()        
              optimizer.step()
  #Accuracy 
  def Test(test_dl, model):
      predictions, actuals = list(), list()
      for i, (inputs, targets) in enumerate(test_dl):
          inputs, targets=inputs.to(device), targets.to(device)
          yhat = model(inputs)        
          yhat =(yhat.cpu()).detach().numpy()
          actual = (targets.cpu()).numpy()
          actual = actual.reshape((len(actual), 1))         
          yhat = yhat.round()
     
          predictions.append(yhat)
          actuals.append(actual)
      predictions, actuals = vstack(predictions), vstack(actuals)
      
      acc = accuracy_score(actuals, predictions)
      print("Accuracy: ",acc)
      return acc


  def Predict(row, model):
     
      row = Tensor([row])     
      yhat = model(row)
      yhat = yhat.detach().numpy()
      return yhat


#Finds all possible combinations of hyperparameters within the specified range
#orders the best combinations sequential order after training 

class Hyperparameters:

   #Control Hyperparameters here
   def __init__(self):

     #Declare Range of Hyperparameters to Generate within 
     #eg: min layers=4 max layers=22 or min n=100 max n=1000
     ####
     #layers=[4,10]
     n=[100,10000]
     d=[2,10]
     self.max_nodes=20
     self.max_combinations_w=15
     self.increment_w=.05
     ####

     #layers_cmb= np.arange(layers[0],layers[1],1)
     n_cmb=np.arange(n[0],n[1],100)
     d_cmb=np.arange(d[0],d[1],1)


     #all possible combinations of hyperparameters 
     self.params=list(itertools.product(n_cmb,d_cmb))
     self.param=[]
     for i in self.params:
       i=list(i)
       i.insert(0,4)
       self.param.append(i)


   def initiate(self,*kwargs): 

     #Train model for every combinations of hyperparameters 
     #record parameters and accuracy, order [layers,n,d,nodes,accuracy]
     self.data=list()   

     for i in self.param:  
             
       #n combinations of nodes
       if i[0]==2:
         val=list()
         val.insert(0,i[2])
         val.append(1)
         val=[val]
         print(val)
       else:
         val=list()
         r=np.arange((i[2]+1),(self.max_nodes),1)
         A= sum([list(map(list, itertools.combinations(r, m))) for m in range(len(r) + 1)], [])  
         val=list()   
         for r in A:
           if len(r)==(i[0]-3):
             r.insert(0,i[2])
             r.append(1)
             val.append(r)

       #n combinations of w
       for j in val:
         a,b=i[2]//2,i[2]%2         
         c=np.ones(a)*-1
         d=np.ones(a)
         if b != 0:
           c=np.append(c,-1)
         sign=[]
         for n in c.tolist():sign.append([n])
         for n in d.tolist():sign.append([n])
         param=(itertools.permutations(sign))
         val=np.arange(0+(self.increment_w),(1+(self.increment_w)),self.increment_w)
         val=np.append(val,(val)*-1)
         n_w=False
         w_cmb=[]
         while n_w==False:
           a=np.random.choice(val,i[2])                       
         
           if np.sum(a)==0:
             if list(a) in w_cmb:               
               continue 
             else:
               w_cmb.append(list(a))

           if len(w_cmb)==self.max_combinations_w:
             n_w=True    
         w_cmb=np.array(w_cmb)         
         
         #training loop
         for k in w_cmb:
           train_dl, test_dl = DataSet.data(n=i[1],d=i[2],W=k)
           print("hidden layers:",(i[0]-3),"n:",i[1],"d:",i[2],"nodes",j,"coefficients",k)           
           model = (Net.MLP(i[0],j)).to(device)
           Main.Train(train_dl, model)   
           acc=Main.Test(test_dl, model) 
           #store layers,n,d,nodes,w,accuracy
           self.data.append([(i[0]-1),i[1],i[2],j,k.tolist(),acc])  
           print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

     return self.data,pd.DataFrame(self.data)

   #Sort Hyperparameters According to accuracy 
   def best_param(self):
     return self.data.sort(key=lambda x: x[4])



hyp=Hyperparameters()
data,pd_data=hyp.initiate()
#save the data as csv file into drive
#change path accordingly if needed
pd_data.to_csv("drive/My Drive/Colab Notebooks/df.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
hidden layers: 1 n: 100 d: 2 nodes [2, 3, 1] coefficients [-0.8  0.8]
Sequential(
  (0): Linear(in_features=2, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
  (3): Sigmoid()
)
Accuracy:  0.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
hidden layers: 1 n: 100 d: 2 nodes [2, 3, 1] coefficients [-0.9  0.9]
Sequential(
  (0): Linear(in_features=2, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
  (3): Sigmoid()
)
Accuracy:  0.5
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
hidden layers: 1 n: 100 d: 2 nodes [2, 3, 1] coefficients [ 0.4 -0.4]
Sequential(
  (0): Linear(in_features=2, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
  (3): Sigmoid()
)
Accuracy:  0.35
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
hidden layers: 1 n: 1